Ahora vamos a ir directo a la accion

Creamos un nuevo proyecto

```dotnetcli
dotnet new webapi -o WebApi
```

Vamos a hacer un boceto de una API para que Pipe pueda cargar sus platos.

- Agregar un menu para una fecha
- Agregar platos para ese menu
- Listar menus y platos para un rango de fechas
- Anotarse para un plato
- Desanotarse para un plato
- Iniciar Sesion 
- Registrarse (Solo usuarios que se anotan a platos)

Primero agregamos el paquete necesario para soportar la base de datos que queramos usar:

```dotnetcli
dotnet add package Npgsql.EntityFrameworkCore.PostgreSQL
```

Creamos las clases de los modelos y agregamos los controllers correspondientes con sus respectivos metodos (podemos hacerlo por rutas al igual que en express). 

```csharp
[Route("Menus")]
public class MenusController : Controller
{
    public MenusController()
    {
    }

    [HttpGet]
    [Route("")]
    public IActionResult Get()
    {
        return Ok(this.Db.Products);
    }
}
```


Creamos una clase que herede de DbContext con sus respectivos DbSet

```C#
public class AppDbContext : DbContext 
{
    public AppDbContext(DbContextOptions<AppDbContext> options) : base(options)
    {
    }

    public DbSet<Menu> Menus {get;set;}
    public DbSet<Dish> Dishes {get;set;}
}
```

Agregamos al archivo <b>Program.cs</b> la configuracion del connection string de la base de datos

```csharp
var connectionString = builder.Configuration.GetConnectionString("Default");

builder.Services.AddDbContext<AppDbContext>(options =>
            {
                options.UseNpgsql(connectionString);
            });
```

Agregamos el metodo para configurar los modelos con la API (dentro del Context)

```csharp
protected override void OnModelCreating(ModelBuilder modelBuilder)
{
    base.OnModelCreating(modelBuilder);
}
```

Creamos las configuraciones de los distintos modelos:

```csharp
public class MenusConfiguration : IEntityTypeConfiguration<Menu>
{
    public void Configure(EntityTypeBuilder<Menu> builder)
    {
        builder.ToTable("Menus"); // Esto no es necesario porque ya sabe que la tabla se llama asi
        // Add Soft Delete Filter
        builder.HasQueryFilter(p => !p.IsDeleted);

        builder.HasData(new Menu[]{
            new Menu{} // Hay que ponerle el Id a mano
        });
    }
}
```

Agregamos la configuracion al AppDbContext en OnModelCreating

```csharp
protected override void OnModelCreating(ModelBuilder modelBuilder)
{
    base.OnModelCreating(modelBuilder);
    // Agregamos aqui
    modelBuilder.ApplyConfiguration(new MenuConfiguration());
    modelBuilder.ApplyConfiguration(new DishConfiguration());
    modelBuilder.ApplyConfiguration(new UserConfiguration());
}
```

Una vez modelado todo esto podemos agregar nuestra primer migracion

```dotnetcli
dotnet ef migrations add InitialCreate
```

Ahora agreguemos la autenticacion

Agregamos un record para la configuracion del JWT

```csharp
public record JwtConfiguration(string ValidAudience, string ValidIssuer, string Secret);
```

En el appsettings agregamos la configuracion del JWT

```json
"JWT": {  
    "ValidAudience": "http://localhost:5006",  
    "ValidIssuer": "http://localhost:5006",  
    "Secret": "ByYM000OLlMQG6VVVp1OH7Xzyr7gHuw1qvUC5dcGt3SNM"  
} 
```

Definimos un metodo de extension para encapsular todas las configuraciones de Autenticacion necesarias:

```csharp
public static class AppAuthConfiguration
{
    public static void AddAppAuthConfig(this WebApplicationBuilder builder)
    {
        var jwtConfiguration = builder.Configuration.GetSection("JWT").Get<JwtConfiguration>();
        if (jwtConfiguration is null)
        {
            throw new Exception("JWT Configuration cannot be empty");
        }
        builder.Services.AddSingleton<JwtConfiguration>(jwtConfiguration);
        
        builder.Services.AddAuthorization();
        builder.Services.AddAuthentication(options =>
        {
            options.DefaultAuthenticateScheme = JwtBearerDefaults.AuthenticationScheme;
            options.DefaultChallengeScheme = JwtBearerDefaults.AuthenticationScheme;
            options.DefaultScheme = JwtBearerDefaults.AuthenticationScheme;
        }).AddJwtBearer(options =>
            {
                options.SaveToken = true;
                // options.RequireHttpsMetadata = false;  
                options.TokenValidationParameters = new TokenValidationParameters()
                {
                    ValidateIssuer = true,
                    ValidateAudience = true,
                    ValidAudience = jwtConfiguration.ValidAudience,
                    ValidIssuer = jwtConfiguration.ValidIssuer,
                    IssuerSigningKey = new SymmetricSecurityKey(Encoding.UTF8.GetBytes(jwtConfiguration.Secret))
                };
            });

        builder.Services
            .AddIdentityCore<IdentityUser>()
            .AddRoles<IdentityRole>()
            .AddDefaultTokenProviders()
            .AddEntityFrameworkStores<AppDbContext>();

        builder.Services.Configure<IdentityOptions>(options =>
        {
            // Password settings.
            options.Password.RequireDigit = true;
            options.Password.RequireLowercase = true;
            options.Password.RequireNonAlphanumeric = true;
            options.Password.RequireUppercase = true;
            options.Password.RequiredLength = 8;
            options.Password.RequiredUniqueChars = 1;

            // Lockout settings.
            options.Lockout.DefaultLockoutTimeSpan = TimeSpan.FromMinutes(5);
            options.Lockout.MaxFailedAccessAttempts = 5;
            options.Lockout.AllowedForNewUsers = true;

            // User settings.
            options.User.AllowedUserNameCharacters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-._@+";
            options.User.RequireUniqueEmail = false;
        });
    }
}
```

Definamos un metodo de extension para configurar las tablas involucradas:

```csharp
public static class IdentityDataBuilder
{
    public static void ConfigureIdentityTables(this ModelBuilder modelBuilder)
    {
        modelBuilder.Entity<IdentityUser>(b => {
            b.ToTable("Users");
        });
        modelBuilder.Entity<IdentityRole>(b => {
            b.ToTable("Roles");
        });
        modelBuilder.Entity<IdentityUserRole<string>>(b => {
            b.ToTable("UserRoles");
        });
        modelBuilder.Entity<IdentityUserToken<string>>(b => {
            b.ToTable("UserTokens");
        });

        modelBuilder.Entity<IdentityRoleClaim<string>>(b => {
            b.ToTable("RoleClaims");
        });
        modelBuilder.Entity<IdentityUserClaim<string>>(b => {
            b.ToTable("UserClaims");
        });
        modelBuilder.Entity<IdentityUserLogin<string>>(b => {
            b.ToTable("UserLogins");
        });
    }
}
```

Luego en el Program ponemos:

```csharp
builder.AddAppAuthConfig();
```

Agreguemos ademas usuarios con ciertos Roles (Administradores) y que los endpoints de registro solo dejen crear usuarios comunes